In [ ]:
import requests
import json
import matplotlib.pyplot as plt
from dateutil import parser
import datetime

# API (Application Programming Interface) Intro

There are two basic ways of accessing data via APIs
* Using raw HTTP requests
* Using client library (essentially just abstracts away the http requests and parsing)

Considerations:
* We take advantage of free open APIs here, typically you'd need to authenticate for API usage (at least the more valuable ones).
* Some APIs have "discovery documents" associated with them, this allows you to understand how data is fetched and organized.

If you'd like to learn about authenticating here is a few good starting apis:
* (Twitter) https://developer.twitter.com/en/docs/developer-utilities/twitter-libraries.html
* (Open weather Map) https://openweathermap.org/api
* (Google static maps, free up to a certain number of requests a month) https://developers.google.com/maps/documentation/maps-static/intro 
    * (Getting an API Key) https://developers.google.com/maps/documentation/maps-static/get-api-key



We got the idea for these examples from here: 
https://github.com/toddmotto/public-apis



## Raw Requests
To do this we take advantage of the requests library: https://pypi.org/project/requests/

### Make the request

In [ ]:
# "Discovery Document: https://open.canada.ca/data/en/dataset/d38de914-c94c-429b-8ab1-8776c31643e3"
url = 'http://healthycanadians.gc.ca/recall-alert-rappel-avis/api/recent/en'
res = requests.get(url)

### Check Status Code 
https://en.wikipedia.org/wiki/List_of_HTTP_status_codes

In [ ]:
print(res.status_code)

### Print out content

In [ ]:
print(res.content)

### Organize Content (Parse JSON Response into python dictionary)

In [ ]:
res_dict = res.json()

print(json.dumps(res_dict, indent=4, sort_keys=True))

In [ ]:
for recall in res_dict['results']["ALL"]:
    print(recall["title"] + '\n')

### More complicated request
Here we add parameters to the base url (essentially feeding variables to whatever function the provider uses to generate the data)

In [ ]:
# "Discovery Document": https://www.coindesk.com/api

def plot_bitcoin_price(start="2017-01-01",end=datetime.datetime.now().strftime('%Y-%m-%d')):
    url = 'https://api.coindesk.com/v1/bpi/historical/close.json'
    
    params = {"index":"USD",
         "currency":"CAD",
         "start":start,
         "end":end}
    
    res = requests.get(url,params)
    
    if res.status_code == 200:
        print('successful request')
    else:
        print('failed request')
        return 1
    
    data = res.json()['bpi']
    
    t = [parser.parse(date) for date in data]
    p = [data[date] for date in data]
    
    fig,ax = plt.subplots(1,1,figsize=(20,20))

    ax.set_title(f"Bitcoin price from {params['start']} to {params['end']}")
    ax.set_ylabel('Price of bitcoin (CAD)')
    ax.set_xlabel('Date')

    ax.plot(t,p)
                 
    return 0

In [ ]:
plot_bitcoin_price()

## Client Library
Here we take advantage of an open source python client for the spaceX launch data API:
https://github.com/r-spacex/SpaceX-API

There appears to be a few clients:
https://github.com/r-spacex/SpaceX-API/blob/master/docs/clients.md

but only two python clients:
* https://github.com/vinayphadnis/SpaceX-Python
* https://github.com/HiKaylum/SpaceX-PY

take a dig through the source code and you'll see it's identical to what we've done above, just
abstracted so we don't need the boilerplate code everytime we make a request to this api.


### Install Client Library via PIP

In [ ]:
# !sudo pip install git+https://github.com/HiKaylum/SpaceX-PY #install from git
!sudo pip install spacex-py #install from pypi

### Import and Get Launch Data

In [ ]:
from spacex_py import launches

In [ ]:
launches, header = launches.get_launches()

### Check out header for fun

In [ ]:
print(json.dumps(dict(header), indent=4, sort_keys=True))

### Look at first launch

In [ ]:
print(json.dumps(launches[0], indent=4, sort_keys=True))

### Plot Aggregate Payload Mass over time

In [ ]:
data = {}


for launch in launches:
    
    total_mass = 0
    
    for payload in launch['rocket']['second_stage']['payloads']:
        mass = payload['payload_mass_kg']
        if mass:
            total_mass += mass
    
    if launch['launch_success']:
        year = launch['launch_year']

        if year not in data:
            data[year] = 0
            data[year] += total_mass
        else:
            data[year] += total_mass

t = [parser.parse(date) for date in data]
m = [data[date] for date in data]
    
            
fig,ax = plt.subplots(1,1,figsize=(20,20))

ax.set_title("Total Payload per Year of Space-X launches")
ax.set_ylabel('Mass(kg)')
ax.set_xlabel('Year')

ax.plot(t,m)
    